# Object Detection: R-FCN

In [ ]:
from __future__ import print_function

import os
import time
import random
import requests
import numpy as np
import tensorflow as tf
from PIL import Image

from object_detection.utils.visualization_utils import visualize_boxes_and_labels_on_image_array

%matplotlib inline
import matplotlib
from matplotlib import pyplot as plt

In [ ]:
SERVER_URL = 'http://localhost:8501/v1/models/rfcn:predict'
IMAGES_PATH = '/home/<user>/coco/val/val2017' # Edit this to your COCO validation directory

In [ ]:
def get_random_image(image_dir):
    image_path = os.path.join(image_dir, random.choice(os.listdir(image_dir)))
    image = Image.open(image_path)
    (im_width, im_height) = image.size
    
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

def visualize(output_dict, image_np):
    output_dict['num_detections'] = int(output_dict['num_detections'])
    output_dict['detection_classes'] = np.array(output_dict['detection_classes']).astype(np.uint8)
    output_dict['detection_boxes'] = np.array(output_dict['detection_boxes'])
    output_dict['detection_scores'] = np.array(output_dict['detection_scores'])

    # Visualize the results of a detection
    visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        {1: {'id': 1, 'name': 'object'}}, # Empty category index
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=8)
    plt.figure()
    plt.imshow(image_np)

# Test Object Detection

In [ ]:
np_image = get_random_image(IMAGES_PATH)

In [ ]:
predict_request = '{"instances" : %s}' % np.expand_dims(np_image, 0).tolist()
result = requests.post(SERVER_URL, data=predict_request)

In [ ]:
visualize(result.json()['predictions'][0], np_image)

# Measure Performance

In [ ]:
def benchmark(batch_size=1, num_iteration=40, warm_up_iteration=10):
    i = 0
    total_time = 0
    for _ in range(num_iteration):
        i += 1
        np_images = np.repeat(np.expand_dims(get_random_image(IMAGES_PATH), 0).tolist(), batch_size, axis=0).tolist()
        predict_request = '{"instances" : %s}' % np_images
        start_time = time.time()
        requests.post(SERVER_URL, data=predict_request)
        time_consume = time.time() - start_time
        print('Iteration %d: %.3f sec' % (i, time_consume))
        if i > warm_up_iteration:
            total_time += time_consume

    time_average = total_time / (num_iteration - warm_up_iteration)
    print('Average time: %.3f sec' % (time_average))
    print('Batch size = %d' % batch_size)
    if batch_size == 1:
        print('Latency: %.3f ms' % (time_average * 1000))
    print('Throughput: %.3f images/sec' % (batch_size / time_average))

## Real-time Inference (latency, batch_size=1)

In [ ]:
benchmark()

## Throughput (batch_size=128)

In [ ]:
benchmark(batch_size=128)